# Hayden French (hmf9kx) Capstone Project

## Overview

This project creates and populates a dimensional data mart modeling sales for a fictional company. It draws data from a SQL (MySQL) database, a NoSQL database (MongoDB), and an API (Reddit).

### API

The API will be used to create the transaction dimension. In order to be able to analyze streaming data I wanted a date corresponding to each transaction. I used the Reddit API to generate pseudo-random transactions for the company, with popular posts corresponding with succesful sales at the company. The date of each transaction can be joined with the date dimension from SQL for later analysis.

In [23]:
# Importing necessary packages
import requests
import pandas as pd
from configparser import ConfigParser
import datetime

In [24]:
# Public credentials
username = 'DS3002API'
client_id = 'no9xCWocGbF-E_D-6U8Meg'

In [25]:
# Reading confidential credentials from a seperate file
parser = ConfigParser()

try:
    _ = parser.read('notebook.cfg')
except:
    print('Unable to read configuration file')
    
try:
    secret_key = parser.get('my_api', 'secret_key')
    pw = parser.get('my_api', 'pw')
except:
    print('Unable to fetch secret key and/or password')

In [26]:
# Authenticating credentials
try:
    auth = requests.auth.HTTPBasicAuth(client_id, secret_key)
except:
    print('Unable to authenticate client id and/or secret key')

In [27]:
data = {
    'grant_type': 'password',
    'username': username,
    'password': pw
}

headers = {'User-Agent': 'My DS3002 API'}

In [28]:
# Retrieving access token from the api
try:
    res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
    token = res.json()['access_token']
except:
    print('Unable to retrieve access token')

In [29]:
headers['Authorization'] = f'bearer {token}'

In [30]:
# Requesting data as a json
try:
    r = requests.get(f'https://oauth.reddit.com/r/news/top', headers=headers, params={'t': 'month', 'limit': '100'}).json()
except:
    print('Unable to fetch posts')

In [31]:
df_transactions = pd.DataFrame(data={'transaction_id': [], 'product_id': [], 'sales_rep_id': [], 'quantity': [], 
                        'date': []}).astype({'transaction_id': int, 'product_id': int, 'sales_rep_id': int,
                                             'quantity': int, 'date': str})

In [32]:
for i in range(0, 100):
    data = r['data']['children'][i]['data']
    transaction_id = i + 1
    unix_time = int(data['created'])
    ups = data['ups']
    product_id = unix_time % 10 + 1
    sales_rep_id = ups % 10 + 1
    quantity = data['total_awards_received'] + 1
    date_time = datetime.datetime.utcfromtimestamp(unix_time).date()
    date = str(date_time).replace("-", "/")
    
    df_transactions.loc[i] = [transaction_id, product_id, sales_rep_id, quantity, date]

In [33]:
df_transactions

,transaction_id,product_id,sales_rep_id,quantity,date
0,1,10,1,281,2022/05/03
1,2,2,1,48,2022/05/04
2,3,7,9,59,2022/05/03
3,4,9,1,25,2022/05/03
4,5,10,3,51,2022/04/18
...,...,...,...,...,...
95,96,9,6,1,2022/04/18
96,97,1,10,12,2022/05/04
97,98,10,10,6,2022/05/03
98,99,9,3,1,2022/04/23


### MySQL

The MySQL database encapsulates the product dimension. I created a fictional product line of 10 food items with corresponsing prices. I also used the provided code to create the date dimension in MySQL. This will later be combined with the date in the transaction database.

In [34]:
from sqlalchemy import create_engine

In [35]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "Passw0rd123"

dbname = "capstone"

In [36]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [47]:
products = "SELECT * FROM capstone.dim_products;"
df_products= get_dataframe(user_id, pwd, host_name, dbname, products)

date = "SELECT * FROM capstone.dim_date"
df_date= get_dataframe(user_id, pwd, host_name, dbname, date)

In [38]:
df_products

,product_key,name,price,in_stock
0,1,Chips,2,1
1,2,Soda,5,1
2,3,Milk,2,1
3,4,Bread,3,1
4,5,Chicken,10,1
5,6,Cereal,6,1
6,7,Eggs,4,1
7,8,Cheese,4,1
8,9,Cookies,5,1
9,10,Candy,2,1


In [48]:
df_date.full_date = df_date.full_date.astype(str)
#CHANGE full_date TO STRING

### NoSQL

The MongoDB database encapsulates the sales representative dimension. I created 10 employees and their corresponding information. Each employee is also either a member of sales team A or sales team B.

In [40]:
import pymongo

In [41]:
host_name = "localhost"
port = 27017

user_id = "root"
pwd = "Passw0rd123"

src_dbname = "capstone"

In [42]:
def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe

In [43]:
query = {}
collection = "sales_reps"

df_sales_reps = get_mongo_dataframe(None, None, host_name, port, src_dbname, collection, query)

In [44]:
df_sales_reps

,sales_rep_key,first_name,last_name,job_title,team,email,phone_number
0,1,Adam,Anderson,Sales Manager,A,adama@gmail.com,4341728463
1,2,Barbara,Blake,Sales Manager,B,barbarab@gmail.com,4342617564
2,3,Cody,Covington,Sales Representative,A,codyc@gmail.com,4342672819
3,4,Dave,Douglass,Sales Representative,B,daved@gmail.com,4341675830
4,5,Evan,Earhart,Sales Representative,A,evane@gmail.com,4341529805
5,6,Frank,Falconer,Sales Representative,B,frankf@gmail.com,4341678923
6,7,Grace,Gaetz,Sales Representative,A,graceg@gmail.com,4347869041
7,8,Hailey,Harrington,Sales Representative,B,haileyh@gmail.com,4345618953
8,9,Igor,Irvin,Sales Representative,A,igori@gmail.com,4347759109
9,10,James,Johnson,Sales Representative,B,jamesj@gmail.com,4347859016


### Exporting from dataframes to jsons

We now have all four of our dataframes processed in Python. Let's export these to .json files so we can easily upload them to Databricks.

In [45]:
import json

In [46]:
df_transactions.to_json('transactions.json', orient='records')
df_products.to_json('products.json', orient='records')
df_date.to_json('date.json', orient='records')
df_sales_reps.to_json('sales_reps.json', orient='records')

### Putting it together in Databricks

The rest of the joining/cleaning will take place in Databricks. Please see the ____ file for the Databricks Notebook. Note that the .json files imported in the notebook are the same files we just exported above. These are all available in the github.

### Visualizations

After creating our final fact_orders table in Databricks, we can create our visualizations in Excel. The exported file from Databricks is "export.csv" but all of the visualizations can be found in "visualizations.xlsx"